In [118]:
import os, json, pprint, re
import pandas as pd
import pickle as pkl
from collections import defaultdict
from matplotlib import pyplot as plt

In [119]:
with open("entries.json", "r") as f:
    mapping = json.load(f)
    
with open("data.pkl", "rb") as f:
    data = pkl.load(f)
    
data = {k: data[k] for k in data.keys()}
data.keys()

dict_keys(['David', 'Gr1', 'Gr2', 'Gr3', 'Gr4', 'Gr5'])

In [120]:
def get_chunk_types(chunk):
    return tuple((".".join(e["phetioID"].split(".")[2:]), e["event"]) for e in chunk)

def flatten(original):
    entry = original.copy()
    children = entry.get("children")
    ret = [entry]
    
    if children:
        entry.pop("children")
        
        for c in children:
            ret += flatten(c)
            
    return ret

def get_chunks_set(data):
    chunks = defaultdict(dict)
    
    for k in data.keys():
        for exp in range(3):
            chunks[k][exp] = [[s for s in flatten(f)] for f in data[k][exp]]
    
    return chunks

In [121]:
# We flatten all chunks together to run tests on our whole dataset
chunks_set = get_chunks_set(data)
flat_chunks = [elem for group, tests in chunks_set.items() for idx, test in tests.items() for elem in test]
x = set([get_chunk_types(c) for c in flat_chunks])
        
elem_to_pos = dict()

for a in x:
    for i, b in enumerate(a):
        try:
            elem_to_pos[b].add(i)
        except:
            elem_to_pos[b] = {i}

In [122]:
pos_view_0 = {e:f for e,f in elem_to_pos.items() if e[0][:4] == "view" and 0 in f}
pos_view_0

{('view.solutionControls.comboBox', 'popupShown'): {0},
 ('view.resetAllButton', 'fired'): {0},
 ('view.cuvetteNode.cuvetteDragHandler', 'dragged'): {0},
 ('view.wavelengthControls.wavelengthSlider.plusButton', 'fired'): {0},
 ('view.cuvetteNode.cuvetteDragHandler', 'dragEnded'): {0},
 ('view.cuvetteNode.cuvetteDragHandler', 'dragStarted'): {0},
 ('view.detectorNode.probeNode.movableDragHandler', 'dragEnded'): {0},
 ('view.solutionControls.comboBox', 'fired'): {0},
 ('view.wavelengthControls.wavelengthSlider.minusButton', 'fired'): {0},
 ('view.wavelengthControls.wavelengthSlider.thumbInputListener',
  'dragEnded'): {0},
 ('view.solutionControls.concentrationControl.slider.thumb.dragHandler',
  'dragged'): {0},
 ('view.solutionControls.concentrationControl.slider.minusButton',
  'fired'): {0},
 ('view.solutionControls.concentrationControl.slider.plusButton',
  'fired'): {0},
 ('view.lightNode.button', 'toggled'): {0},
 ('view.beersLawScreenLargeButton', 'fired'): {0},
 ('view.solutionC

In [123]:
pos_view_1 = {e:f for e,f in elem_to_pos.items() if e[0][:4] == "view" and 0 not in f}
pos_view_1

{('view.detectorNode.bodyNode.valueNode', 'textChanged'): {2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  13,
  14,
  17,
  20},
 ('view.wavelengthControls.valueDisplay', 'textChanged'): {2, 3, 4, 5, 8, 15},
 ('view.wavelengthControls.variableWavelengthProperty', 'changed'): {1,
  14,
  18,
  25},
 ('view.concentrationMeterNode.bodyNode.readoutTextNode', 'textChanged'): {1}}

In [124]:
pos_model = {e:f for e,f in elem_to_pos.items() if e[0][:5] == "model"}
pos_model

{('model.solutionProperty', 'changed'): {1, 4, 11},
 ('model.light.wavelengthProperty', 'changed'): {1, 2, 5, 12},
 ('model.detector.valueProperty', 'changed'): {2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  12,
  13,
  16,
  19},
 ('model.light.onProperty', 'changed'): {1, 8, 11, 18},
 ('model.cuvette.widthProperty', 'changed'): {1, 14, 21},
 ('model.detector.probe.locationProperty', 'changed'): {1, 11, 15, 22},
 ('model.detector.modeProperty', 'changed'): {1, 12, 16, 23},
 ('model.ruler.locationProperty', 'changed'): {1, 13, 17, 24}}

In [125]:
pos_solutions = {e:f for e,f in elem_to_pos.items() if e[0][:9] == "solutions"}
pos_solutions

{('solutions.drinkMix.concentrationProperty', 'changed'): {1},
 ('solutions.nickelIIChloride.concentrationProperty', 'changed'): {1, 2, 8},
 ('solutions.potassiumPermanganate.concentrationProperty', 'changed'): {1,
  3,
  10},
 ('solutions.potassiumChromate.concentrationProperty', 'changed'): {1, 7},
 ('solutions.cobaltIINitrate.concentrationProperty', 'changed'): {1, 2},
 ('solutions.potassiumDichromate.concentrationProperty', 'changed'): {1, 6},
 ('solutions.cobaltChloride.concentrationProperty', 'changed'): {1, 5},
 ('solutions.copperSulfate.concentrationProperty', 'changed'): {1, 9}}

In [126]:
pos_known = set(pos_view_0) | set(pos_view_1) | set(pos_model) | set(pos_solutions)
pos_others = {k : elem_to_pos[k] for k in set(elem_to_pos) - set(pos_known)}
pos_others

{('', 'invoked'): {0},
 ('', 'simStarted'): {0},
 ('screenIndexProperty', 'changed'): {1},
 ('showHomeScreenProperty', 'changed'): {1}}

In [127]:
# This entry is suspicious. Here we check for which root nodes it
# appears and we can deduce that it indeed represents a display.

s = 'view.wavelengthControls.variableWavelengthProperty'
[y[0] for y in x if s in tuple(zip(*y))[0] and y[0][0] != s]

[('view.resetAllButton', 'fired'),
 ('view.resetAllButton', 'fired'),
 ('view.resetAllButton', 'fired'),
 ('view.wavelengthControls.presetWavelengthRadioButton', 'fired'),
 ('view.wavelengthControls.presetWavelengthRadioButton', 'fired'),
 ('view.wavelengthControls.variableWavelengthRadioButton', 'fired'),
 ('view.wavelengthControls.presetWavelengthRadioButton', 'fired')]

In [128]:
sorted(pos_view_0)

[('view.beersLawScreenLargeButton', 'fired'),
 ('view.beersLawScreenSmallButton', 'fired'),
 ('view.cuvetteNode.cuvetteDragHandler', 'dragEnded'),
 ('view.cuvetteNode.cuvetteDragHandler', 'dragStarted'),
 ('view.cuvetteNode.cuvetteDragHandler', 'dragged'),
 ('view.detectorNode.bodyNode.absorbanceRadioButton', 'fired'),
 ('view.detectorNode.bodyNode.transmittanceRadioButton', 'fired'),
 ('view.detectorNode.probeNode.movableDragHandler', 'dragEnded'),
 ('view.detectorNode.probeNode.movableDragHandler', 'dragStarted'),
 ('view.detectorNode.probeNode.movableDragHandler', 'dragged'),
 ('view.lightNode.button', 'toggled'),
 ('view.resetAllButton', 'fired'),
 ('view.rulerNode.movableDragHandler', 'dragEnded'),
 ('view.rulerNode.movableDragHandler', 'dragStarted'),
 ('view.rulerNode.movableDragHandler', 'dragged'),
 ('view.solutionControls.comboBox', 'fired'),
 ('view.solutionControls.comboBox', 'popupHidden'),
 ('view.solutionControls.comboBox', 'popupShown'),
 ('view.solutionControls.concent